In [ ]:
# Run this cell.
from lec_utils import *
plotly.io.renderers.default = 'notebook'

def show_cv_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vTydTrLDr-y4nxQu1OMsaoqO5EnPEISz2VYmM6pd83ke8YnnTBJlp40NfNLI1HMgoaKx6GBKXYE4UcA/embed?start=false&loop=false&delayms=60000&rm=minimal"
    display(IFrame(src, width=900, height=361))
    
from sklearn.datasets import load_breast_cancer

full = load_breast_cancer()
df = pd.DataFrame(full['data'], columns=full['feature_names'])
df['target'] = 1 - full['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], random_state=23, stratify=df['target'])

def format_results(searcher):
    cv = sum([1 if ('split' in s and '_test_score' in s) else 0 for s in list(searcher.cv_results_.keys())])
    deg = searcher.cv_results_['mean_fit_time'].shape[0]
    
    df = pd.DataFrame(np.vstack([searcher.cv_results_[f'split{i}_test_score'] for i in range(cv)]))
    df.index = [f'Fold {i}' for i in range(1, cv + 1)]
    df.columns = [f'p={j}' for j in range(1, deg + 1)]
    return df

#### DAIR-3 Workshop, Day 2 • Building Robust ML Models

# Part 3: Model Selection

**Instructor**: Suraj Rampure (rampure@umich.edu)

### Outline

- Cross-validation.
- Other considerations.
    - Regularization.
    - Feature standardization.
    - Data leakage.

## Cross-validation

---

### Choosing between models

- Often, we want to choose between several different candidate models.
    - Logistic regression using all 30 features, or just 29, 28, 27, ...
    - Logistic regression using PCA with $p = 2$, or $p = 3$, or $p = 4$...
    - Decision tree with a depth of 3, or 4, or 5, or 10...

    <br>

    The value of $p$ in PCA, or the depth of a decision tree, is a **hyperparameter** of the model. Think of hyperparameters as **knobs** 🎛️ we get to tune.

- **One idea**: Train each candidate model on the training set, compute test set accuracy, and pick the model with the best test set accuracy.

<center><img src="images/train-test-first.png" width=600></center>

- **Why is this problematic?**

- Because, we are **overfitting to the test set** – the best hyperparameter for the test set might not be the best hyperparameter for a totally unseen dataset.

### Idea: A single validation set

<center><img src='images/train-test-val.png' width=600></center>

1. Split the data into three sets: <span style='color: blue'><b>training</b></span>, <span style='color: green'><b>validation</b></span>, and <span style='color: orange'><b>test</b></span>.

2. For each candidate model, <span style='color: blue'><b>train</b></span> the model only on the <span style='color: blue'><b>training set</b></span>, and <span style='color: green'><b>evaluate</b></span> the model's performance on the <span style='color: green'><b>validation set</b></span>.

3. Find the candidate with the best <span style='color: green'><b>validation</b></span> performance (e.g. highest accuracy).

4. Retrain the final model on the <span style='color: blue'><b>training</b></span> and <span style='color: green'><b>validation</b></span> sets, and report its performance on the <span style='color: orange'><b>test set</b></span>.

- **Issue**: This strategy is too dependent on the <span style='color: green'><b>validation</b></span> set, which may be small and/or not a representative sample of the data. **We're not going to do this.** ❌

### A better idea: $k$-fold cross-validation

- Instead of relying on a single <span style='color: green'><b>validation</b></span> set, we can create $k$ <span style='color: green'><b>validation</b></span> sets, where $k$ is some positive integer (5 in the example below).

<center><img src='images/k-fold.png' width=500></center>

- Since each data point is used for <span style='color: blue'><b>training</b></span> $k-1$ times and <span style='color: green'><b>validation</b></span> once, the (averaged) <span style='color: green'><b>validation</b></span> performance should be a good metric of a model's ability to generalize to unseen data.


- $k$-fold cross-validation (or simply "cross-validation") is **the** gold standard for choosing between candidate models.

<div class="alert alert-danger"><h3>Warning #4: Lack of Cross-Validation</h3>
    
Be weary of studies that choose models based on their performance on a single test set, as their test set performance will be overly optimistic.

While test set performance may be all that is reported, verify that cross-validation was used to actually select the model.

### Illustrating $k$-fold cross-validation

- To illustrate $k$-fold cross-validation, let's use the following example dataset with $n = 12$ rows.<br><small>Suppose this dataset represents our **training set**, i.e. suppose we already performed a train-test split.</small>

In [ ]:
training_data = pd.DataFrame().assign(x=range(0, 120, 10),
                                      y=[9, 1, 58, 3, 6, 4, -2, 8, 1, 10, 1.1, -45])        

display_df(training_data, rows=12)

- Suppose we choose $k = 4$. Then, each fold has $\frac{12}{4} = 3$ rows.

In [ ]:
show_cv_slides()

### Modeling pipelines

- Let's return to the breast cancer dataset.<br>In the previous notebook, we used a scree plot to choose the number of principal components to use. This did not factor in model performance (e.g. accuracy) at all.

In [ ]:
df

- Let's build a **Pipeline object** that first performs PCA with $p$ components, then fits a logistic regression model. To choose $p$, we'll use cross-validation, choosing the number of components that maximizes average cross-validated accuracy.

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [ ]:
pipe = make_pipeline(PCA(), LogisticRegression()) # Note that we haven't specified n_components!
pipe

### `GridSearchCV`

- `GridSearchCV` takes in:
    - an **un-`fit`** Pipeline object, and
    - a **dictionary** of hyperparameter values to try,
    
  and performs $k$-fold cross-validation to find the **combination of hyperparameters** with the best average validation performance.

In [ ]:
from sklearn.model_selection import GridSearchCV

searcher = GridSearchCV(
    estimator=make_pipeline(PCA(), LogisticRegression()),
    param_grid={
        'pca__n_components': range(1, 31),
    },
    cv=5,
    scoring='accuracy'
)
searcher

In [ ]:
# The orange diagram indicates the estimator (which GridSearchCV is) has yet to be fit.
searcher.fit(X_train, y_train)

- It seems that keeping $p = 5$ principal components leads to the best validation accuracy, so that's what we should do (among the 30 choices; perhaps there are better models to choose).

In [ ]:
searcher.best_params_

### Interpreting the results of $k$-fold cross-validation

- Let's peek under the hood.

In [ ]:
acc_df = format_results(searcher) # Helper function defined at the top of the notebook.
acc_df

- Note that for each choice of $p$ (our hyperparameter), we have **five** accuracies, one for each "fold" of the data. This means that in total, $5 \cdot 30 = 150$ models were trained!

- Remember, our goal is to choose the $p$ with the **highest average** validation accuracy.

In [ ]:
acc_df.mean(axis=0)

In [ ]:
fig = acc_df.mean(axis=0).iloc[:31].plot(kind='line', title='Average Validation Accuracy')
fig.update_layout(xaxis_title='Degree', yaxis_title='Average Validation Accuracy', showlegend=False)

In [ ]:
# Chosen automatically by sklearn.
acc_df.mean(axis=0).idxmax()

- Note that if we didn't perform $k$-fold cross-validation, but instead just used a single validation set, we may have ended up with a different result (so be weary of single-validation-set studies!).

In [ ]:
acc_df.idxmax(axis=1)

### Now that a model has been chosen...

- $p = 5$ is the optimal number of principal components, according to our cross-validation procedure.

- However, it is **not necessarily** the value of $p$ that has the highest test set accuracy! We are **intentionally not looking** at the test set accuracy of each possible $p$.

- Observe:
    - The **test set accuracy** of the $p = 5$ model is **higher** than the all-30-features-but-no-PCA model, but
    - the **training set accuracy** of the $p = 5$ model is **lower**.
    <br>
    
    **Why?** Discuss.

In [ ]:
model_all_features = LogisticRegression()
model_all_features.fit(X_train, y_train)
model_all_features.score(X_test, y_test)

print(f'p = 5 PCA, then Logistic Regression\nTraining Accuracy: {round(searcher.score(X_train, y_train), 4)} \t Testing Accuracy: {round(searcher.score(X_test, y_test), 4)}\n\n')
print(f'All 30 Features Logistic Regression\nTraining Accuracy: {round(model_all_features.score(X_train, y_train), 4)} \t Testing Accuracy: {round(model_all_features.score(X_test, y_test), 4)}')

### Training and test set accuracy vs. model complexity

(See annotated slides.)

<div class="alert alert-warning">
    <h3>Question 🤔</h3>
        
- Suppose you have a training dataset with 1000 rows.
- You want to decide between 20 hyperparameters for a particular model.
- To do so, you perform 10-fold cross-validation.
- **How many times is the first row in the training dataset (`X.iloc[0]`) used for training a model?**


### Choosing between many models

- Often, we want to use cross-validation to choose between different model types, not just different hyperparameter values.<br>Doing so is slightly more complicated in code, but the general idea is the same.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer

pipes = {
    'Logistic Regression (All 30 Features)': LogisticRegression(),
    'Logistic Regression (Mean Radius Only)': make_pipeline(
        FunctionTransformer(lambda X: X[['mean radius']], validate=False), 
        LogisticRegression()
    ),
    'Logistic Regression (5 PCA Components)': make_pipeline(PCA(n_components=5), LogisticRegression()),
    'Decision Tree (Depth 3)': DecisionTreeClassifier(max_depth=3, random_state=23),
    'Decision Tree (Depth 10)': DecisionTreeClassifier(max_depth=10, random_state=23),
    'Random Forest (100 Estimators)': RandomForestClassifier(n_estimators=100, random_state=23),
}

- Here, we will have to call `GridSearchCV` multiple times.

In [ ]:
results = pd.DataFrame(columns=['Average Training Accuracy', 'Average Validation Accuracy'])

for pipe in pipes:
    fitted = GridSearchCV(
        pipes[pipe],
        param_grid={}, # No hyperparameters, but we could have them.
        scoring='accuracy',
        cv=5, # Change this and see what happens!
        return_train_score=True # So that we can compute training accuracies, too.
    )
    
    fitted.fit(X_train, y_train)
    results.loc[pipe] = [fitted.cv_results_['mean_train_score'][0], fitted.cv_results_['mean_test_score'][0]]
    
cancer_models_summarized = (
    results
    .sort_values('Average Training Accuracy')
    .plot(kind='barh', barmode='group', width=1000)
    .update_layout(xaxis_title='Accuracy', yaxis_title='Model')
)

cancer_models_summarized

- Which model is most likely to perform best in practice? Which model has the highest bias? Variance?

### Summary: Model selection

- We care about how well our models **generalize** to unseen data.<br><small><ul><li>The more complex a model is, the more it will **overfit** to the noise in the training data, and have high **model variance**.</li><li>The less complex a model is, the more it will **underfit** the training data, and have high **bias**.</li></ul>

- To choose the model that is **most likely to generalize well**:
    1. Split the data into two sets: <span style='color: blue'><b>training</b></span> and <span style='color: orange'><b>test</b></span>.
    2. Use only the <span style='color: blue'><b>training</b></span> data when designing, training, and tuning the model.
        - Use <span style='color: green'><b>$k$-fold cross-validation</b></span> to choose between candidate models and to estimate a model's ability to generalize.
        - Do not ❌ look at the <span style='color: orange'><b>test</b></span> data in this step!
    3. Commit to your final model and train it using the entire <span style='color: blue'><b>training</b></span> set.
    4. Test the data using the <span style='color: orange'><b>test</b></span> data. If the performance (e.g. accuracy) is not acceptable, return to step 2.
    5. Finally, train on **all available data** and ship the model to production! 🛳

- 🚨 This is the process you should **always** use! 🚨 

### What's next?

Let's look at a few common sources of error in the model selection process:

- Regularization and logistic regression.
- Feature standardization.
- Data leakage.

## Other considerations

---

### Read the documentation!

**Activity**: Look at the documentation for `sklearn.linear_model.LogisticRegression` by running the cell below (or going [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)). Identify **1 unexpected operation** that it performs, without explicitly saying so.

In [ ]:
LogisticRegression?

For context, read the documentation for the `LinearRegression` class as well (see [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)).

In [ ]:
from sklearn.linear_model import LinearRegression
LinearRegression?

<div class="alert alert-danger"><h3>Warning #5: Unreported Regularization</h3>
    
Logistic regression in `sklearn` applies $L_2$ regularization by default, using a regularization penalty of $\lambda = 1$. 
    
<br>
    
There are some (arguably good) reasons for this, but keep this in mind when using it (and when reviewing others' work that uses logistic regression!).
    
<br>
    
If you do want to use regularization, use `LogisticRegressionCV` or `GridSearchCV` to choose a regularization hyperparameter – **don't** just use the default. To turn off regularization, set `C=np.inf` as an argument in `LogisticRegression`.

</div>

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">By default, logistic regression in scikit-learn runs w L2 regularization on and defaulting to magic number C=1.0. How many millions of ML/stats/data-mining papers have been written by authors who didn&#39;t report (&amp; honestly didn&#39;t think they were) using regularization?</p>&mdash; Zachary Lipton (@zacharylipton) <a href="https://twitter.com/zacharylipton/status/1167298276686589953?ref_src=twsrc%5Etfw">August 30, 2019</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

### Feature standardization

- **Some** models benefit from having standardized features, in which, separately for each feature (column), we apply the transformation:

    $$x_i \rightarrow \frac{x_i - \bar{x}}{\sigma_x}$$

    where $\bar{x}$ and $\sigma_x$ are the mean and standard deviation of the column, respectfully.

In [ ]:
from sklearn.preprocessing import StandardScaler
make_pipeline(StandardScaler(), PCA(n_components=5), LogisticRegression())

- **Discuss**: When might we standardize, and why?

Run the cell below to set up the next visualization.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer

pipes = {
    'Logistic Regression (All 30 Features)': LogisticRegression(),
    'Logistic Regression (All 30 Features, Standardized)': make_pipeline(StandardScaler(), LogisticRegression()),
    'Logistic Regression (All 30 Features, No Regularization)': LogisticRegression(C=np.inf),
    'Logistic Regression (All 30 Features, Standardized, No Regularization)': make_pipeline(StandardScaler(), LogisticRegression(C=np.inf)),
    'Logistic Regression (5 PCA Components)': make_pipeline(PCA(n_components=5), LogisticRegression()),
    'Logistic Regression (5 PCA Components, Standardized)': make_pipeline(StandardScaler(), PCA(n_components=5), LogisticRegression()),
    'Decision Tree (Depth 3)': DecisionTreeClassifier(max_depth=3, random_state=23),
    'Decision Tree (Depth 3, Standardized)': make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=3, random_state=23)),
    'Decision Tree (Depth 10)': DecisionTreeClassifier(max_depth=10, random_state=23),
    'Decision Tree (Depth 10, Standardized)': make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=10, random_state=23)),
    'Random Forest (100 Estimators)': RandomForestClassifier(n_estimators=100, random_state=23),
    'Random Forest (100 Estimators, Standardized)': make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, random_state=23)),
}

results = pd.DataFrame(columns=['Training Accuracy'])

for pipe in pipes:
    model = pipes[pipe]
    model.fit(X_train, y_train)
    train_acc = model.score(X_train, y_train)
    # test_acc = model.score(X_test, y_test)
    results.loc[pipe] = [train_acc]

# Define color pairs for each model type (raw/std, no_reg/std_no_reg, etc.)
# There are 6 pairs, so 6 colors, each repeated twice for the pair
import matplotlib.pyplot as plt

color_pairs = [
    "#1f77b4", "#1f77b4",  # raw_features, std_raw_features
    "#ff7f0e", "#ff7f0e",  # raw_features_no_reg, std_raw_features_no_reg
    "#2ca02c", "#2ca02c",  # pca_5, std_pca_5
    "#d62728", "#d62728",  # dt_depth_3, std_dt_depth_3
    "#9467bd", "#9467bd",  # dt_depth_10, std_dt_depth_10
    "#8c564b", "#8c564b",  # rf_n_estimators_100, std_rf_n_estimators_100
]

# The results are indexed by the keys in pipes, so we need to reverse the color list if we reverse the DataFrame
reversed_colors = color_pairs[::-1]

std_model_comparison = (
    results
    .iloc[::-1]
    .plot(kind='barh', color=reversed_colors, width=1000)
)

std_model_comparison = std_model_comparison.update_layout(showlegend=False, xaxis_title='Training Accuracy', yaxis_title='Model')

### Comparison of model performance, with and without standardization

In [ ]:
std_model_comparison

### Leakage

- **Data leakage** occurs when information about the test set is used to train a model, even in the presence of a train-test split and cross-validation. This will lead to overly optimistic estimates of model performance.

- **Discuss**: How might data leakage occur? Give examples in the context of patient data.

<br><br><br><br>

<div class="alert alert-danger"><h3>Warning #6: Leakage with Feature Standardization</h3>
    
If performing feature standardization, do so AFTER performing a train-test split, not before! If you use `StandardScaler` as part of a Pipeline object that you `fit` directly, this is handled for you.
    
If you standardize features before splitting the data, then the means and variances of the test set are encoded in the training set, and the test set won't be a truly fresh sample of unseen data, and test set performance will be overly optimistic.
    
</div>

### Leakage and feature standardization

- **Good**:

In [ ]:
# Here, features are standardized using only the information in X_train.
# When model.predict is called, features are transformed using the means and variances
# of the columns in X_train.

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)
model.score(X_test, y_test)

- **Bad**:

In [ ]:
without_y = df.iloc[:, :-1]
features_std = StandardScaler().fit_transform(without_y)
# Often, this is implemented manually, e.g.
# features_std = (without_y - without_y.mean(axis=0)) / without_y.std(axis=0)

# X_train, X_test, y_train, y_test = train_test_split(features_std, df.iloc[:, -1], random_state=23, stratify=df.iloc[:, -1])